# sources:

## audio preparation:
* https://www.tensorflow.org/tutorials/audio/simple_audio
* https://www.tensorflow.org/io/tutorials/audio

## siamese network:
* https://github.com/hlamba28/One-Shot-Learning-with-Siamese-Networks

## misc:
* https://gitlab.tu-berlin.de/dl4aed/dl4aed-lectures/blob/master/04-audio-preprocessing.ipynb

In [ ]:
%%bash
# install and update required packages
python3 -m pip install --upgrade pip -q
python3 -m pip install -r requirements.txt -q
python3 -m pip install --no-deps tensorflow-io==0.17.0

In [ ]:
%reload_ext autoreload
%autoreload 2

In [ ]:
''' Small hack to restart kernel after netbook ran to free GPU memory '''
## restart jupyter kernel to free all memory in notebook
from IPython.display import display_html
def restartkernel():
    display_html("<script>Jupyter.notebook.kernel.restart()</script>",raw=True)

In [ ]:
''' Some imports '''
import tensorflow as tf
assert tf.__version__ >= "2.4.0"
import numpy as np
from matplotlib import pyplot as plt
%matplotlib inline

In [ ]:
''' Allow typical dynamic GPU memory allocation and read config'''
from utils import load_config, save_config, allow_growth
allow_growth()
# read the config file
# it includes more or less all hyperparameter used in the model and preprocessing/training step
data_parameter, model_parameter, training_parameter = load_config(verbose=0)

In [ ]:
''' Parsing CLI arguments and overwriting config '''
from argument_parser import parse_arguments, overwrite_config
parsed_arguments, _ = parse_arguments()
overwrite_config(parsed_arguments, data_parameter, model_parameter, training_parameter)

In [ ]:
''' Some small amount of reproducibility '''
tf.random.set_seed(training_parameter["seed"])
np.random.seed(training_parameter["seed"])

In [ ]:
''' Create model and save config '''
from network import Network
network = Network(model_parameter, training_parameter)
network.compile()
network.save()
# save config into model path
save_config(data_parameter, model_parameter, training_parameter, network)

In [ ]:
''' Load data and prepare datasets '''
from data_generator import create_train_data, create_test_data
train_dataset = create_train_data(training_parameter)
test_dataset = create_test_data(training_parameter)

In [ ]:
''' Training procedure '''
AUTOTUNE = tf.data.experimental.AUTOTUNE

network.fit(train_dataset, epochs=training_parameter["epochs"],
                   initial_epoch=network.epoch, max_queue_size=AUTOTUNE,
                   workers=AUTOTUNE, use_multiprocessing=False, callbacks=network.callbacks)

In [ ]:
''' Evaluation procedure '''
network.evaluate(test_dataset)

In [ ]:
''' Current junkyard '''
# from sklearn.metrics import confusion_matrix, multilabel_confusion_matrix, classification_report, ConfusionMatrixDisplay

# some minor visual inspection
for ((x1, x2), label) in test_dataset:
    for x1_, x2_, label_ in zip(x1,x2,label):
        plt.imshow(x1_)
        plt.show()
        plt.imshow(x2_)
        plt.show()
        print(label_.numpy())
        print(tf.cast(tf.round(
            network.predict_on_batch([tf.expand_dims(x1_, axis=0),tf.expand_dims(x2_, axis=0)])
        ), dtype=tf.bool).numpy())
    break

In [ ]:
# restart kernel to free GPU mem
restartkernel()